In [ ]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

try:
    import apyori
except:
    !pip install apyori

from apyori import apriori

# Data Exploration

In [ ]:
loan = pd.read_csv(r'/kaggle/input/personal-loan/personal_loan.csv')

In [ ]:
loan.head(2)

In [ ]:
plt.subplot(2, 2, 1)
loan.save_act.value_counts().plot(kind='pie')
plt.subplot(2, 2, 2)
loan.current_act.value_counts().plot(kind='pie')
plt.subplot(2, 2, 3)
loan.mortgage.value_counts().plot(kind='pie')
plt.subplot(2, 2, 4)
loan.car.value_counts().plot(kind='pie')

In [ ]:
plot = loan.children.value_counts().plot(kind='bar')
plot.set_xlabel("Number of Children", labelpad=20, weight='bold', size=12)
plot.set_ylabel("Number of Applicants", labelpad=20, weight='bold', size=12)
plot

In [ ]:
loan.married.value_counts().plot(kind='pie')

In [ ]:
plot = loan.income.hist(bins=6)
plot.set_xlabel("Yearly Income", labelpad=20, weight='bold', size=12)
plot.set_ylabel("Number of Applicants", labelpad=20, weight='bold', size=12)
plot

In [ ]:
loan.sex.value_counts().plot(kind='pie')

In [ ]:
loan.region.value_counts().plot(kind='pie')

In [ ]:
plot = loan.age.hist(bins=6)
plot.set_xlabel("Age (years)", labelpad=20, weight='bold', size=12)
plot.set_ylabel("Number of Applicants", labelpad=20, weight='bold', size=12)
plot

# Data Prep

In [ ]:
loan.income = pd.qcut(loan.income, 4, labels=['very_low_income','low_income','medium_income','high_income'])
loan.age = pd.qcut(loan.age, 3, labels=['young','middle_age','old'])
loan.drop(['id'], axis=1, inplace=True)

loan.married.replace({'NO': 'not_married', 'YES': 'married'}, inplace=True)
loan.children.replace({0:'no_children',1:'1_child',2:'2_children',3:'3_children'}, inplace=True)
loan.car.replace({'NO': 'no_car', 'YES': 'car'}, inplace=True)
loan.save_act.replace({'NO': 'no_savings_act', 'YES': 'savings_act'}, inplace=True)
loan.current_act.replace({'NO': 'no_act', 'YES': 'act'}, inplace=True)
loan.mortgage.replace({'NO': 'no_mortgage', 'YES': 'mortgage'}, inplace=True)
loan.pep.replace({'NO': 'no_pep', 'YES': 'pep'}, inplace=True)

In [ ]:
loan.head(2)

## Creating Association Rules

In [ ]:
rules = list(apriori(loan.values,
        min_support=0.12,
        min_confidence=0.80,
        min_lift=1.8,
        max_length=None))

In [ ]:
for rule in rules:
    stat = rule.ordered_statistics[0]
    print('Rule: ', list(stat.items_base), ' -> ', list(stat.items_add))
    print("Support: " + str(rule[1]))

    print("Confidence: " + str(stat.confidence))
    print("Lift: " + str(stat.lift))
    print("=====================================")

## Finding Association Rules specifically tied to PEP

In [ ]:
rules = list(apriori(loan.values,
        min_support=0.05,
        min_confidence=0.7,
        min_lift=2,
        max_length=3))

In [ ]:
for rule in rules:
    stat = rule.ordered_statistics[0]
    if (list(stat.items_add)[0] not in ['pep','no_pep']) or len(stat.items_add) > 1:
        continue
    
    print('Rule: ', list(stat.items_base), ' -> ', list(stat.items_add))
    print("Support: " + str(rule[1]))

    print("Confidence: " + str(stat.confidence))
    print("Lift: " + str(stat.lift))
    print("=====================================")